In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv("data/hius.csv")
df_2 = pd.read_csv("data/quickcharts_hius.csv")

#the first row is the description of the column so we should drop that as well
df = df.drop(0, axis=0)
df.head()

In [ ]:
df_2 = df_2.drop(0, axis=0)
df_2.head()

In [ ]:
# the difference between the two datasets
list(set(df.columns) - set(df_2.columns))

In [ ]:
for column in df.columns:
    print("{} has values:\n{}\n".format(column, df[column].unique()))
    print("-"*100)

In [ ]:
#figuring out how many nans per column
print(df.shape)
df.isna().sum()

In [ ]:
#separating diseases

disList = df['GHO (CODE)'].unique()

diseases = []
for i in disList:
    tempDis = df[df['GHO (CODE)'] == i]
    diseases.append(tempDis)
    
#nan values for each disease df
for i in range(len(diseases)):
    print(diseases[i].head())

# for i in diseases[1]:
    
#     x = i.values[:, :-1]

#     nan_count = i.isna().sum()
#     nan_percentage = [count / x.shape[0] for count in nan_count]
#     columns = i.columns

#     threshold = 0.6

#     # And let's plot those
#     fig1 = plt.figure(figsize=(30,15))
#     ax = fig1.add_subplot(111)
#     plt.axhline(y=threshold,linewidth=1, color="k")
#     ax.bar(columns, nan_percentage)
#     plt.xticks(rotation=90)
#     ax.set_xlabel('Column')
#     ax.set_ylabel('NaN Count')
#     ax.set_title('Columns with missing values')
    

### Conclusion:

The set is as good as complete, but some columns only apply to some diseases/conditions
So, we will remove all unimportant columns for each disease.

In [ ]:
for i in diseases:
    x = i.values[:, :-1]

    nan_count = i.isna().sum()
    nan_percentage = [count / x.shape[0] for count in nan_count]
    columns = i.columns

    threshold = 0.6

    too_much_cols = []
    for j in range(len(columns)):
        if nan_percentage[j] >= threshold:
            diseases[i] = diseases[i].drop(column[j], axis=1, inplace=True)
    
diseases[0].head()

In [ ]:
print(diseases[0].dtypes)

All values are objects, change to numeric values.

In [ ]:

for i in diseases:
    for j in i['YEAR (CODE)']:
#         print(j, ' ', type(j))
#         if not int(j):
            print('calling')
#             strAr = j.split('-')
#             a = int(strAr[0])
#             b = int(strAr[1])
#             c = int((b-a)/2)
#             j = c
#         else:
#             j = int(j)
# sns.lineplot(x='YEAR (CODE)', y='Display Value', data=diseases[0])